# Part 7b - Opportunity Provider interaction with Youth


## 1. Initialise a controller for Yoma Opportunity Provider Sub Wallet 1

In [28]:
%autoawait
import time
import asyncio
from termcolor import colored,cprint

from aries_basic_controller.aries_controller import AriesAgentController
    
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 8022
WEBHOOK_BASE = ""
ADMIN_URL = "http://yoma-multi-tenant-agent:8021"
%store -r op1_token

# Based on the aca-py agent you wish to control
agent_controller = AriesAgentController(webhook_host=WEBHOOK_HOST, webhook_port=WEBHOOK_PORT,
                                       webhook_base=WEBHOOK_BASE, admin_url=ADMIN_URL, jwt_token=op1_token)
    

Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7f3e63192760>


IPython autoawait is `on`, and set to use `asyncio`


## 2. Register Listeners

The handler should get called every time the controller receives a webhook with the topic issue_credential, printing out the payload. The agent calls to this webhook every time it receives an issue-credential protocol message from a credential.

In [29]:
loop = asyncio.get_event_loop()
loop.create_task(agent_controller.listen_webhooks())

def cred_handler(payload):
    print("Handle Credentials")
    exchange_id = payload['credential_exchange_id']
    state = payload['state']
    role = payload['role']
    attributes = payload['credential_proposal_dict']['credential_proposal']['attributes']
    print(f"Credential exchange {exchange_id}, role: {role}, state: {state}")
    print(f"Offering: {attributes}")
    
cred_listener = {
    "topic": "issue_credential",
    "handler": cred_handler
}

def connections_handler(payload):
    global STATE
    connection_id = payload["connection_id"]
    print("Connection message", payload, connection_id)
    STATE = payload['state']
    if STATE == 'active':
#         print('Connection {0} changed state to active'.format(connection_id))
        print(colored("Connection {0} changed state to active".format(connection_id), "red", attrs=["bold"]))


connection_listener = {
    "handler": connections_handler,
    "topic": "connections"
}

agent_controller.register_listeners([cred_listener,connection_listener], defaults=True)


Task exception was never retrieved
future: <Task finished name='Task-45' coro=<AriesAgentController.listen_webhooks() done, defined at /aries_basic_controller/aries_controller.py:140> exception=OSError(98, "error while attempting to bind on address ('0.0.0.0', 8022): address already in use")>
Traceback (most recent call last):
  File "/aries_basic_controller/aries_controller.py", line 147, in listen_webhooks
    await self.webhook_site.start()
  File "/opt/conda/lib/python3.8/site-packages/aiohttp/web_runner.py", line 121, in start
    self._server = await loop.create_server(
  File "/opt/conda/lib/python3.8/asyncio/base_events.py", line 1463, in create_server
    raise OSError(err.errno, 'error while attempting '
OSError: [Errno 98] error while attempting to bind on address ('0.0.0.0', 8022): address already in use


## 8a. Check for any existing connections

In [30]:
# Check for existing connections
connection = await agent_controller.connections.get_connections()
print("EXISTING CONNECTIONS")
for key, value in connection.items():
    for item in value:
        print('ConnectionID:', item['connection_id'], 'Status:',item['state'])

EXISTING CONNECTIONS
ConnectionID: 161107bc-b4b9-418f-b70d-a30ac2db98be Status: active
ConnectionID: df3ce906-5201-4884-ac1e-ad907c34cd38 Status: active
ConnectionID: f8dc691c-cbe1-407f-a599-7df871b0630d Status: invitation


## 8b. Create an Invitation


In [5]:
# Create Invitation
invite = await agent_controller.connections.create_invitation()
connection_id = invite["connection_id"]
print("Connection ID", connection_id)
print("Invitation")
print(invite['invitation_url'])
inviteURL = invite['invitation_url']
invitation = invite['invitation']
print('\n ',invitation)

Connection ID f8dc691c-cbe1-407f-a599-7df871b0630d
Invitation
http://677011df58cb.ngrok.io?c_i=eyJAdHlwZSI6ICJkaWQ6c292OkJ6Q2JzTlloTXJqSGlxWkRUVUFTSGc7c3BlYy9jb25uZWN0aW9ucy8xLjAvaW52aXRhdGlvbiIsICJAaWQiOiAiYmY4Y2YwMGUtOGE5Mi00OGJmLWJmMTUtNzQ5ZTdiNGUzOGMzIiwgInNlcnZpY2VFbmRwb2ludCI6ICJodHRwOi8vNjc3MDExZGY1OGNiLm5ncm9rLmlvIiwgInJlY2lwaWVudEtleXMiOiBbIjM0UmhaWmh6TE1HYThXblkyTUxvazNyTFhlaUp2THo5VGJtdldkdkNKajVFIl0sICJpbWFnZVVybCI6ICJodHRwczovL2FyaWVzLmNhL2ltYWdlcy9zYW1wbGUucG5nIiwgImxhYmVsIjogIllvbWEgT3Bwb3J0dW5pdHkgUHJvdmlkZXIgMSJ9

  {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation', '@id': 'bf8cf00e-8a92-48bf-bf15-749e7b4e38c3', 'serviceEndpoint': 'http://677011df58cb.ngrok.io', 'recipientKeys': ['34RhZZhzLMGa8WnY2MLok3rLXeiJvLz9TbmvWdvCJj5E'], 'imageUrl': 'https://aries.ca/images/sample.png', 'label': 'Yoma Opportunity Provider 1'}


In [31]:
## TODO DELETE THIS CELL - JUST USED FOR TEST
connection_id = 'df3ce906-5201-4884-ac1e-ad907c34cd38'

### 13. Check if connection state is active

It should be if you send a trust ping from Alice's side

In [32]:
# Print connection list
connection = await agent_controller.connections.get_connection(connection_id)
print("Youth Opportunity Provider 1 AGENT CONNECTION")
print(connection)
print("State:", connection["state"])

Youth Opportunity Provider 1 AGENT CONNECTION
{'created_at': '2021-03-16 15:38:12.018422Z', 'invitation_mode': 'once', 'connection_id': 'df3ce906-5201-4884-ac1e-ad907c34cd38', 'routing_state': 'none', 'updated_at': '2021-03-16 15:38:36.432253Z', 'my_did': 'JpGtE9KmwdX3pLRR2KKUy7', 'state': 'active', 'invitation_key': 'BRGR2YLXufLQB9jysudMiYP9HASbPVBdW4b1zx2zLPmY', 'accept': 'auto', 'their_role': 'invitee', 'their_label': 'Yoma Youth1', 'rfc23_state': 'completed', 'their_did': 'Nm2Vg3KH2FMZeicGwyZese'}
State: active


## 3a. Use a previously written schemas


### Youth Member Schema

In [ ]:
schema_id = '3pMbbpqEEGzSQkV79jGmDH:2:Youth Member:0.0.1'

### Ecosystem Partner Schema aka Opportunity Provider

In [ ]:
schema_id = '3pMbbpqEEGzSQkV79jGmDH:2:Ecosystem Member:0.0.1'

### Opportunity Provider Challenge Schema

In [18]:
schema_id = 'LCQvtKCEvm4yM2K3yA9uNT:2:Youth Challenge Credential:0.0.1'

## 3b. (OPTIONAL) Write a Schema to the Ledger

For more details see the [schema-api notebook](http://localhost:8888/notebooks/schema_api.ipynb)

**Note: You will only be able to do this once unless you change the schema_name or version. Once you have a schema written to the ledger it can be reused by multiple entities**

In [ ]:
# Define you schema name - must be unique on the ledger
schema_name = "SSI PyDentity Tutorial"
# Can version the schema if you wish to update it
schema_version = "0.0.1"
# Define any list of attributes you wish to include in your schema
attributes = ["fullname", "skill", "age"]

response = await agent_controller.schema.write_schema(schema_name, attributes, schema_version)
schema_id = response["schema_id"]
print(schema_id)


## 4. Write a Credential Definition to the Ledger

**Note: Again this can only be done once per issuer, per schema_id.**

In [19]:
response = await agent_controller.definitions.write_cred_def(schema_id)

cred_def_id = response["credential_definition_id"]
print(cred_def_id)

XVRRjbJuz9qczLQoBPmifF:3:CL:194453:default


**Note: You should be able to see both schema and definition transactions on Sovrin Stagingnet network with [Indyscan](https://indyscan.io/home/SOVRIN_STAGINGNET)**

## 5. Populate the Attribues to Issue to Idenity Holder (User)

We will issue a credential to the identity holder consisting of the following attributes:

credential_attributes = [
    {"name": "fullname", "value": name},
    {"name": "skill", "value": "PyDentity SSI Ninja"},
    {"name": "age", "value": age}
]

The notebook will ask you to input the identity holder's full name and age which will be used to populate the schema above with the identity holders attribute information.

In [20]:
name=input("Please enter challenge name: ")
completed=input("Please enter if youth completed challenge: (True|False)")
url=input("Please enter challenge url: ")
credential_attributes = [
    {"name": "challenge_name", "value": name},
    {"name": "challenge_completed", "value": completed},
    {"name": "challenge_url", "value": url}
]
print(credential_attributes)

Please enter challenge name: COVID-19
Please enter if youth completed challenge: (True|False)True
Please enter challenge url: www.cartedo.com
[{'name': 'challenge_name', 'value': 'COVID-19'}, {'name': 'challenge_completed', 'value': 'True'}, {'name': 'challenge_url', 'value': 'www.cartedo.com'}]


## 9. Send Credential

This sends a credential to a identity holder (User), and automates the rest of the protocol.

There are other ways to issue a credential that require multiple api calls.

**Arguments**
* connection_id: The connection_id of the holder you wish to issue to (MUST be in active state)
* schema_id: The id of the schema you wish to issue
* cred_def_id: The definition (public key) to sign the credential object. This must refer to the schema_id and be written to the ledger by the same public did that is currently being used by the agent.
* attributes: A list of attribute objects as defined above. Must match the schema attributes.
* comment (optional): Any string, defaults to ""
* auto_remove (optional): Boolean, defaults to True. I believe this removes the record of this credential once it has been issued. (TODO: double check)
* trace (optional): Boolean, defaults to False. **Not entirely sure about this one, believe its for logging. Also when set to True it throws an error**

In [33]:
record = await agent_controller.issuer.send_credential(connection_id, schema_id, cred_def_id, credential_attributes, trace=False)
record_id = record['credential_exchange_id']
state = record['state']
role = record['role']
print(f"Credential exchange {record_id}, role: {role}, state: {state}")


Credential exchange e990d61c-59fc-41f4-a450-39fd084145fd, role: issuer, state: offer_sent


## 10. Accept credential in [Youth Side Notebook](http://localhost:8888/notebooks/Part%207a%20-%20Youth%20Side%20-%20Youth%20and%20Opportunity%20Provider%20interaction.ipynb)

## 11. End of Tutorial

Be sure to terminate the controller so you can run another tutorial.


In [27]:
response = await agent_controller.terminate()
print(response)

RuntimeError: Site <aiohttp.web_runner.TCPSite object at 0x7f3e631864a0> is not registered in runner <aiohttp.web_runner.AppRunner object at 0x7f3e63184cc0>